# Credit card analysis

The following notebook consists of tasks completed aiming to provide insights on credit card usage of a certain bank. The mentioned tasks have been completed with additional visualisations shown where possible. The final output is stored in an excel file with all results of all the tasks stored in form of multiple sheets.

In [1]:
# Import libraries

import pandas as pd
import numpy as np
import calendar
import matplotlib.pyplot as plt
import plotly.express as px

Reading and cleaning data if necessary

In [2]:
# Read data
customers = pd.read_excel('task1_dataset.xls', sheet_name='Customer Acqusition')
spend = pd.read_excel('task1_dataset.xls', sheet_name='Spend')
repayment = pd.read_excel('task1_dataset.xls', sheet_name='Repayment')

In [3]:
customers.head()

,Sl No:,Customer,Age,City,Credit Card Product,Limit,Company,Segment
0,1,A1,0.928521,BANGALORE,Gold,500000,C1,Self Employed
1,2,A2,35.534551,CALCUTTA,Silver,100000,C2,Salaried_MNC
2,3,A3,11.559307,COCHIN,Platimum,10000,C3,Salaried_Pvt
3,4,A4,45.820278,BOMBAY,Platimum,10001,C4,Govt
4,5,A5,69.663948,BANGALORE,Platimum,10002,C5,Normal Salary


In [4]:
customers.describe()

,Sl No:,Age,Limit
count,100.000000,100.000000,100.000000
mean,50.500000,40.146505,284700.610000
std,29.011492,23.629594,218494.967852
min,1.000000,0.726493,10000.000000
25%,25.750000,20.254138,100000.000000
50%,50.500000,40.545682,300001.500000
75%,75.250000,61.606910,500000.000000
max,100.000000,78.431756,500000.000000


In [5]:
customers.notnull().count()

Sl No:                 100
Customer               100
Age                    100
City                   100
Credit Card Product    100
Limit                  100
Company                100
Segment                100
dtype: int64

In [6]:
spend.head()

,Sl No:,Costomer,Month,Type,Amount
0,1,A1,2004-01-12,JEWELLERY,473775.834869
1,2,A1,2004-01-03,PETRO,335578.666019
2,3,A1,2004-01-15,CLOTHES,371040.941912
3,4,A1,2004-01-25,FOOD,141177.813256
4,5,A1,2005-01-17,CAMERA,398403.771812


In [7]:
spend.describe()

,Sl No:,Amount
count,1500.000000,1500.000000
mean,750.500000,254234.698954
std,433.157015,145872.919043
min,1.000000,783.032154
25%,375.750000,121631.602551
50%,750.500000,262959.087543
75%,1125.250000,381925.385450
max,1500.000000,499682.470459


In [8]:
spend.notnull().count()

Sl No:      1500
Costomer    1500
Month       1500
Type        1500
Amount      1500
dtype: int64

In [9]:
repayment.head()

,SL No:,Costomer,Month,Amount
0,1,A1,2004-01-12,331844.007374
1,2,A1,2004-01-03,441139.807335
2,3,A1,2004-01-15,32480.004009
3,4,A1,2004-01-25,90636.966534
4,5,A1,2005-01-17,1581.969829


In [10]:
repayment.describe()

,SL No:,Amount
count,1500.000000,1500.000000
mean,750.500000,247472.296084
std,433.157015,143530.648485
min,1.000000,183.218203
25%,375.750000,120422.872372
50%,750.500000,247332.616560
75%,1125.250000,369014.873206
max,1500.000000,499344.940170


In [11]:
repayment.notnull().count()

SL No:      1500
Costomer    1500
Month       1500
Amount      1500
dtype: int64

In [12]:
# Rename columns

customers = customers.rename(columns={'Sl No:' : 'SNo'})
spend = spend.rename(columns={
    'Sl No:' : 'SNo',
    'Costomer' : 'Customer',
    'Month' : 'Date'
})
repayment = repayment.rename(columns={
    'SL No:' : 'SNo',
    'Costomer' : 'Customer',
    'Month' : 'Date'
})

In [13]:
# Credit cards are not issued to customers aged below 18. So normalising
# the data by setting age to 18 for customers aged below 18.

customers['Age'] = np.where(customers['Age'] < 18, 18, customers['Age'])
customers.head()

,SNo,Customer,Age,City,Credit Card Product,Limit,Company,Segment
0,1,A1,18.000000,BANGALORE,Gold,500000,C1,Self Employed
1,2,A2,35.534551,CALCUTTA,Silver,100000,C2,Salaried_MNC
2,3,A3,18.000000,COCHIN,Platimum,10000,C3,Salaried_Pvt
3,4,A4,45.820278,BOMBAY,Platimum,10001,C4,Govt
4,5,A5,69.663948,BANGALORE,Platimum,10002,C5,Normal Salary


### Task 1 - Spendings

- Calculating monthly spend of each customer
- Monthly spend for bank every month over the years
- Check if any customer has spent more than the monthly limit of their card

In [14]:
# Extract month and year from date for grouping

month_spend = pd.DatetimeIndex(spend["Date"]).month
year_spend = pd.DatetimeIndex(spend["Date"]).year
temp_spend = list()

for i in range(0, len(month_spend)):
    temp_spend.append(f'{month_spend[i]}-{year_spend[i]}')

spend['Month'] = temp_spend
spend.head()

,SNo,Customer,Date,Type,Amount,Month
0,1,A1,2004-01-12,JEWELLERY,473775.834869,1-2004
1,2,A1,2004-01-03,PETRO,335578.666019,1-2004
2,3,A1,2004-01-15,CLOTHES,371040.941912,1-2004
3,4,A1,2004-01-25,FOOD,141177.813256,1-2004
4,5,A1,2005-01-17,CAMERA,398403.771812,1-2005


In [15]:
# Calculate monthly spend for each customer
monthly_spend = spend.groupby(['Customer', 'Month']).sum().round(3).drop(columns=['SNo']).sort_values(by=['Customer', 'Month'])
monthly_spend.head()

Amount
Customer Month               
A1       1-2004   1511172.746
         1-2005    398403.772
         10-2006   220735.009
         11-2005   457316.899
         2-2004     41381.106

In [16]:
# Calculate and visualise monthly spend for the bank
monthly_spend_bank = spend.groupby(['Month']).sum().round(3).drop(columns=['SNo'])
monthly_spend_bank.index = pd.to_datetime(monthly_spend_bank.index)
monthly_spend_bank['month'] = pd.DatetimeIndex(pd.DataFrame(monthly_spend_bank.index)['Month']).month_name()
monthly_spend_bank['year'] = pd.DatetimeIndex(pd.DataFrame(monthly_spend_bank.index)['Month']).year
monthly_spend_bank = monthly_spend_bank.sort_index()

# Plot graph

fig_monthly_spend_bank = px.histogram(
    monthly_spend_bank,
    x='month',
    y='Amount',
    color='year',
    barmode='group',
    title = 'Monthly spendings from 2004-2006'
)
fig_monthly_spend_bank.show()

In [17]:
# Check if any customer has spent more than the credit limit in any month

monthly_spend_cust_check = pd.merge(monthly_spend.reset_index(), customers, on='Customer', how='left')
monthly_spend_cust_check = monthly_spend_cust_check[['Customer', 'Month', 'Amount', 'Limit']]

def check_spend(amount, limit):
    if amount > limit:
        return True
    else:
        return False

monthly_spend_cust_check['Spend check'] = monthly_spend_cust_check.apply(lambda x: check_spend(x.Amount, x.Limit), axis=1)
customers_cross_limit = monthly_spend_cust_check.loc[(monthly_spend_cust_check['Spend check'] == True)]
customers_cross_limit

,Customer,Month,Amount,Limit,Spend check
0,A1,1-2004,1511172.746,500000,True
5,A1,2-2005,1404193.042,500000,True
6,A1,4-2006,564506.857,500000,True
9,A10,1-2004,747428.092,500000,True
16,A10,5-2005,696067.279,500000,True
...,...,...,...,...,...
796,A9,9-2004,324844.693,100003,True
803,A96,1-2004,320363.536,100000,True
804,A97,1-2004,164330.042,10000,True
805,A98,1-2004,87483.512,10001,True


### Task 2 - Repayment
- Calculating monthly repayment of each customer
- Calculating and visualising monthly repayment for the bank

In [18]:
# Extract month and year from the repayment date

month_repayment = pd.DatetimeIndex(repayment["Date"]).month
year_repayment = pd.DatetimeIndex(repayment["Date"]).year
temp_repayment = list()

for i in range(0, len(month_repayment)):
    temp_repayment.append(f'{month_repayment[i]}-{year_repayment[i]}')

repayment['Month'] = temp_repayment
repayment.head()

,SNo,Customer,Date,Amount,Month
0,1,A1,2004-01-12,331844.007374,1-2004
1,2,A1,2004-01-03,441139.807335,1-2004
2,3,A1,2004-01-15,32480.004009,1-2004
3,4,A1,2004-01-25,90636.966534,1-2004
4,5,A1,2005-01-17,1581.969829,1-2005


In [19]:
# Calculate monthly repayment

monthly_repayment = repayment.groupby(['Customer', 'Month']).sum().round(3).drop(columns=['SNo']).sort_values(by=['Customer', 'Month'])
monthly_repayment.head()

Amount
Customer Month               
A1       1-2004   1362775.235
         1-2005      1581.970
         11-2005   278486.416
         11-2006    17931.399
         2-2004    191180.012

In [20]:
# Calculate and visualise monthly repayment for the bank
monthly_repayment_bank = repayment.groupby(['Month']).sum().round(3).drop(columns=['SNo'])
monthly_repayment_bank.index = pd.to_datetime(monthly_repayment_bank.index)
monthly_repayment_bank['month'] = pd.DatetimeIndex(pd.DataFrame(monthly_repayment_bank.index)['Month']).month_name()
monthly_repayment_bank['year'] = pd.DatetimeIndex(pd.DataFrame(monthly_repayment_bank.index)['Month']).year
monthly_repayment_bank = monthly_repayment_bank.sort_index()

# Plot graph

fig_monthly_repayment_bank = px.histogram(
    monthly_repayment_bank,
    x='month',
    y='Amount',
    color='year',
    barmode='group',
    title = 'Monthly repayment from 2004-2006'
)
fig_monthly_repayment_bank.show()

### Task 3
- Showing top 10 highest paying customers

In [21]:
# Use monthly repayment data to calcualte top 10

top10_repayments = repayment.groupby(['Customer']).sum().round(3).drop(columns=['SNo']).sort_values(by=['Amount'], ascending=False)
top10_repayments.head(10)

,Amount
Customer,
A22,9767170.683
A60,9262032.375
A61,8807888.339
A40,8805084.898
A47,8529825.865
A43,8458621.332
A48,8432803.656
A41,8374045.758
A49,8259840.547


### Task 4
- Finding segments in which people are spending the most

In [22]:
# Use the spend data to calcualte the top spendings category

top_spend_category = spend.groupby(['Type']).count().drop(columns=['SNo', 'Date', 'Amount', 'Month', 'Date']).sort_values(by=['Customer'], ascending=False)
top_spend_category

,Customer
Type,
PETRO,200
CAMERA,160
FOOD,160
AIR TICKET,147
TRAIN TICKET,132
SHOPPING,113
BUS TICKET,99
CLOTHES,95
JEWELLERY,95


In [23]:
# Pie chart representation of all the categories

fig_category = px.pie(
    top_spend_category,
    values='Customer',
    names= top_spend_category.index,
    title='Amount spent based on categories'
)
fig_category.show()

### Task 5
- Finding which age group is spending the most

In [24]:
# Use the spend data to calcualte overall spend of the customers
# followed by joining it into the customers data to find the age 
# and finally aggregating the spendings.

total_spend_customer = spend.groupby(['Customer']).sum().round(3).drop(columns=['SNo'])
customers_spendings = pd.merge(customers, total_spend_customer, on='Customer', how='inner')
customers_spendings.head()

,SNo,Customer,Age,City,Credit Card Product,Limit,Company,Segment,Amount
0,1,A1,18.000000,BANGALORE,Gold,500000,C1,Self Employed,4858294.331
1,2,A2,35.534551,CALCUTTA,Silver,100000,C2,Salaried_MNC,3151947.444
2,3,A3,18.000000,COCHIN,Platimum,10000,C3,Salaried_Pvt,2871164.306
3,4,A4,45.820278,BOMBAY,Platimum,10001,C4,Govt,3121668.966
4,5,A5,69.663948,BANGALORE,Platimum,10002,C5,Normal Salary,4816555.209


In [25]:
# Create age group col where age group will be defined. There
# will be 4 age groups as follows:
#   1. 18-24
#   2. 24-35
#   3. 35-60 
#   4. 60+

def assignAgeGroup(age):
    '''Function to assign age group as per customer age'''
    if  18 <= age <= 24:
        return '18-24'
    elif 24 <= age <= 35:
        return '24-35'
    elif 35 <= age <= 60:
        return '35-60'
    elif 60 < age:
        return '60+'
    
customers_spendings['Age Group'] = customers['Age'].apply(assignAgeGroup)
customers_spendings.head()

,SNo,Customer,Age,City,Credit Card Product,Limit,Company,Segment,Amount,Age Group
0,1,A1,18.000000,BANGALORE,Gold,500000,C1,Self Employed,4858294.331,18-24
1,2,A2,35.534551,CALCUTTA,Silver,100000,C2,Salaried_MNC,3151947.444,35-60
2,3,A3,18.000000,COCHIN,Platimum,10000,C3,Salaried_Pvt,2871164.306,18-24
3,4,A4,45.820278,BOMBAY,Platimum,10001,C4,Govt,3121668.966,35-60
4,5,A5,69.663948,BANGALORE,Platimum,10002,C5,Normal Salary,4816555.209,60+


In [26]:
# Finding the spendings based on each age group
ageGroup_spendings = customers_spendings.groupby(['Age Group']).sum().round(2).drop(columns=['SNo', 'Limit', 'Age']).sort_values(by=['Amount'], ascending=False)
ageGroup_spendings

,Amount
Age Group,
60+,1.340952e+08
18-24,1.134784e+08
35-60,1.070999e+08
24-35,2.667861e+07


In [27]:
# Visualising the spendings based on age group

fig_ageGroupSpendings = px.bar(
    ageGroup_spendings,
    x=ageGroup_spendings.index,
    y='Amount',
    title='Credit card spendings based on age group'
)
fig_ageGroupSpendings.show()

### Task 6
- Finding the segment with most spendings

In [28]:
# Use the customer_spendings data to calculate

segment_spendings = customers_spendings.groupby(['Segment']).sum().round(3).drop(columns=['Age', 'Limit', 'SNo']).sort_values(by=['Amount'], ascending=False)
segment_spendings

,Amount
Segment,
Normal Salary,1.077071e+08
Salaried_Pvt,7.170431e+07
Self Employed,7.097548e+07
Govt,6.732563e+07
Salaried_MNC,6.363949e+07


In [29]:
# Visualising the spendings based on segments

fig_segmentSpendings = px.bar(
    segment_spendings,
    x=segment_spendings.index,
    y='Amount',
    title='Credit card spendings based on employment segments'
)
fig_segmentSpendings.show()

### Task 7
- Imposing interest of 2.7% on due amount

In [30]:
# Calculate the repayed amount by each customer, subtract the repayed
# amount from repayed amount to calulate the due amount and impose
# penalty on the resultant due amount.

repayed_amount = repayment.groupby(['Customer']).sum().round(3).drop(columns=['SNo']).rename(columns={'Amount' : 'Repayed Amount'})
customers_spendings = pd.merge(customers_spendings, repayed_amount, on='Customer', how='inner')
customers_spendings.head()

,SNo,Customer,Age,City,Credit Card Product,Limit,Company,Segment,Amount,Age Group,Repayed Amount
0,1,A1,18.000000,BANGALORE,Gold,500000,C1,Self Employed,4858294.331,18-24,3831937.754
1,2,A2,35.534551,CALCUTTA,Silver,100000,C2,Salaried_MNC,3151947.444,35-60,3653252.167
2,3,A3,18.000000,COCHIN,Platimum,10000,C3,Salaried_Pvt,2871164.306,18-24,2055044.098
3,4,A4,45.820278,BOMBAY,Platimum,10001,C4,Govt,3121668.966,35-60,2507424.488
4,5,A5,69.663948,BANGALORE,Platimum,10002,C5,Normal Salary,4816555.209,60+,5125186.888


In [31]:
# Calculate due ammount

due_amount = list()
for cust in range(0, len(customers_spendings)):
    due_amount.append(customers_spendings['Amount'][cust]-customers_spendings['Repayed Amount'][cust])
customers_spendings['Due Amount'] = due_amount
customers_spendings.head()

,SNo,Customer,Age,City,Credit Card Product,Limit,Company,Segment,Amount,Age Group,Repayed Amount,Due Amount
0,1,A1,18.000000,BANGALORE,Gold,500000,C1,Self Employed,4858294.331,18-24,3831937.754,1026356.577
1,2,A2,35.534551,CALCUTTA,Silver,100000,C2,Salaried_MNC,3151947.444,35-60,3653252.167,-501304.723
2,3,A3,18.000000,COCHIN,Platimum,10000,C3,Salaried_Pvt,2871164.306,18-24,2055044.098,816120.208
3,4,A4,45.820278,BOMBAY,Platimum,10001,C4,Govt,3121668.966,35-60,2507424.488,614244.478
4,5,A5,69.663948,BANGALORE,Platimum,10002,C5,Normal Salary,4816555.209,60+,5125186.888,-308631.679


In [32]:
# Impose penalty

def imposePenalty(due_amount : int):
    if due_amount <= 0:
        return 'NA'
    else:
        return (due_amount+0.029*due_amount)

customers_spendings['Due after penalty'] = customers_spendings['Due Amount'].apply(imposePenalty)
customers_spendings.head()

,SNo,Customer,Age,City,Credit Card Product,Limit,Company,Segment,Amount,Age Group,Repayed Amount,Due Amount,Due after penalty
0,1,A1,18.000000,BANGALORE,Gold,500000,C1,Self Employed,4858294.331,18-24,3831937.754,1026356.577,1056120.917733
1,2,A2,35.534551,CALCUTTA,Silver,100000,C2,Salaried_MNC,3151947.444,35-60,3653252.167,-501304.723,NA
2,3,A3,18.000000,COCHIN,Platimum,10000,C3,Salaried_Pvt,2871164.306,18-24,2055044.098,816120.208,839787.694032
3,4,A4,45.820278,BOMBAY,Platimum,10001,C4,Govt,3121668.966,35-60,2507424.488,614244.478,632057.567862
4,5,A5,69.663948,BANGALORE,Platimum,10002,C5,Normal Salary,4816555.209,60+,5125186.888,-308631.679,NA


In [33]:
customer_dues = customers_spendings.drop(
    columns=['SNo', 'Age Group']
)
customer_dues.head()

,Customer,Age,City,Credit Card Product,Limit,Company,Segment,Amount,Repayed Amount,Due Amount,Due after penalty
0,A1,18.000000,BANGALORE,Gold,500000,C1,Self Employed,4858294.331,3831937.754,1026356.577,1056120.917733
1,A2,35.534551,CALCUTTA,Silver,100000,C2,Salaried_MNC,3151947.444,3653252.167,-501304.723,NA
2,A3,18.000000,COCHIN,Platimum,10000,C3,Salaried_Pvt,2871164.306,2055044.098,816120.208,839787.694032
3,A4,45.820278,BOMBAY,Platimum,10001,C4,Govt,3121668.966,2507424.488,614244.478,632057.567862
4,A5,69.663948,BANGALORE,Platimum,10002,C5,Normal Salary,4816555.209,5125186.888,-308631.679,NA


In [34]:
# Save final outputs into excel files

with pd.ExcelWriter('creditCardAnalysis.xlsx') as writer:
    # Monthly spend of each customer
    monthly_spend.to_excel(writer, sheet_name='Spendings-Monthly', index=True)

    # Monthly repayment of each customer
    monthly_repayment.to_excel(writer, sheet_name='Repayment-Monthly', index=True)

    # Top 10 highest paying customers
    top10_repayments.to_excel(writer, sheet_name='Top 10 Customers', index=True)

    # Categories with most transactions
    top_spend_category.to_excel(writer, sheet_name='Top categories', index=True)

    # Age group spendings
    ageGroup_spendings.to_excel(writer, sheet_name='Spendings-Age Group', index=True)

    # Employment segment spendings
    segment_spendings.to_excel(writer, sheet_name='Spendings-Employment Segment', index=True)

    # Customer dues
    customer_dues.to_excel(writer, sheet_name='Cutomer Dues', index=True)
    
    # Customers with spendings more than monthly limit
    customers_cross_limit.to_excel(writer, sheet_name='Customers-Above limit', index=True)

    # Monthly spendings of the bank
    monthly_spend_bank.to_excel(writer, sheet_name='Spendings-Monthly bank', index=True)

    # Monthly repayment of bank
    monthly_repayment_bank.to_excel(writer, sheet_name='Repayment-Monthly Bank', index=True)